# Dollar Spot Model Forecaster
## Using the Smith-Kerns Dollar Spot Prediction Model and Forecasted Weather Data to predict occurence of Dollar Spot.

### The Dollar Spot Model is an equation developed by Dr. Damon Smith and Dr. Jim Kerns and peer-reviewed and validated through PLoS One (http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0194216)

#### The equation for the model is:
Logit (μ) = -11.4041 + (0.0894 * MeanRH) + (0.1932 * MeanAT)

where: 
 
MeanRH = average relative humidity over a 5 day period

MeanAT = average air temperature (in Celsius) over a 5 day period

#### The logit (μ) is then inserted into the following equation to give the probability

$$ P = e^{logit(μ)} / (1 + e^{logit(μ)} ) * 100 $$

where:

P = Probability of a dollar spot epidemic

e = Euler's number (~2.718)

##### Import relevant libraries and tools

In [65]:
import csv
import requests
import json
import math
from IPython.core.display import display, HTML

##### Import and assign API keys

In [78]:
keys = []

In [110]:
with open('keys.csv') as key:
    readCSV = csv.reader(key, delimiter=',')    
    for row in readCSV:
        keys.append(row)

In [80]:
dk_key = keys[0]
geo_key = keys[1]

##### Enter zipcode to determine lat & long

In [81]:
zip_code = input()

32789


##### Use Google Geocode API to determine approximate co-ords

In [82]:
geo_call = "https://maps.googleapis.com/maps/api/geocode/json?address=" + str(zip_code) + "&key=" + str(geo_key[0])

In [83]:
geo_resp = requests.get(geo_call)
geo_data = geo_resp.json()
lat = geo_data['results'][0]['geometry']['location']['lat']
long = geo_data['results'][0]['geometry']['location']['lng']

##### Use Dark Sky API to determine forecast

In [84]:
dk_url = "https://api.darksky.net/forecast/"

In [85]:
exclude = "?exclude=currently,minutely,hourly,alerts"

In [86]:
units = "&units=si"

In [87]:
dk_call = dk_url + dk_key[0] + "/" + str(lat) + "," + str(long) + exclude + units

In [112]:
dk_resp = requests.get(dk_call)
dk_data = dk_resp.json()
h_temps = []
l_temps = []
avg_temps = []
hums = []

##### Get high temps, low temps, and humidity for every day in the forecast

In [90]:
for x in dk_data['daily']['data']:
    h_temps.append(x['temperatureHigh'])
    l_temps.append(x['temperatureLow'])
    hums.append(x['humidity'])

##### Combine the high temps and low temps to get a list of average temps

In [91]:
for i in range(0, len(h_temps)):
    avg_temps.append((h_temps[i] + l_temps[i])/2)
    
avg_temps

[24.479999999999997,
 25.645,
 24.72,
 24.215,
 24.835,
 26.924999999999997,
 27.744999999999997,
 25.465000000000003]

##### Get average temp & humidity forecast for the upcoming 5 days

In [104]:
avg_hum = sum(hums[:5])/len(hums[:5])
avg_temp = sum(avg_temps[:5])/len(avg_temps[:5])
print(avg_hum)
print(avg_temp)

0.744
24.779000000000003


##### Calculate the logit(μ)

In [105]:
logit_mu = -11.4041 + (0.0894 * avg_hum) + (0.1932 * avg_temp)

In [106]:
logit_mu

-6.550283599999998

##### Calculate the probability

In [109]:
if avg_temp <= 10:
    print("Probability: 0%, average temperature too low")
elif avg_temp >= 35:
    print("Probability: 0%, average temperature too high")
else:
    P = (math.exp(logit_mu) / (1 + math.exp(logit_mu))) * 100
    P = round(P, 2)
    print("Probability: " + str(P) + "%")

Probability: 0.14%


In [18]:
display(HTML('<h1><a href src="https://darksky.net/poweredby/">Powered by Dark Sky</a></h1>'))